In [ ]:
import pandas as pd
import requests
from lxml import html

### Iegūstam veikalu sarakstu

In [ ]:
def get_shops(url, selector):
    """
    """
    doc = requests.get(url)
    tree = html.fromstring(doc.content)
    return [s.lower() for s in tree.xpath(selector)]

In [ ]:
shops = get_shops(
    "https://www.kurpirkt.lv/veikali.php",
    "//ul[@class=\"veikali\"]/li[@class=\"veikals\"]/a/text()"
)
len(shops)

### Iegūstam komentāru sarakstu katram veikalam

In [ ]:
def get_comment_pages(shop, selector):
    """
    """
    url_tpl = "https://www.salidzini.lv/veikals/{}?p={}"
    page = 0
    comment_page = []

    while page == 0 or len(comment_page) > 0:
        page += 1
        doc = requests.get(url_tpl.format(shop, page))
        tree = html.fromstring(doc.content)
        comment_page = tree.xpath(selector)

        yield comment_page

In [ ]:
def get_rating(comment):
    """
    """
    return comment.xpath(
        ".//div[@itemprop=\"reviewRating\"]/meta[@itemprop=\"ratingValue\"]"
    )[0].attrib.get("content")

In [ ]:
def get_rating_date(comment):
    """
    """
    return comment.xpath(
        ".//div[@itemprop=\"datePublished\"]"
    )[0].attrib.get("content")

In [ ]:
def get_rating_body(comment):
    """
    """
    ret = comment.xpath(".//div[@itemprop=\"reviewBody\"]/text()")
    return len(ret) > 0 and ret[0] if isinstance(ret, list) else ret

In [ ]:
data = []
shop_count = 0
comment_count = 0

for shop in shops:
    shop_count += 1

    for comment_page in get_comment_pages(shop, "//table[@class=\"comment_table\"]"):
        for comment in comment_page:
            comment_count += 1
            data.append([
                shop,
                get_rating_date(comment),
                get_rating(comment),
                get_rating_body(comment),
            ])

    print("{}. {}: {}".format(shop_count, shop, comment_count))
    comment_count = 0

In [ ]:
df = pd.DataFrame(data, columns=["Shop", "Date", "Rating", "Comment"])
with pd.ExcelWriter("ratings.xlsx") as writer:
    df.to_excel(writer)